In [1]:
import numpy as np
import pandas as pd
from glob import glob

from osgeo import gdal
import rasterio
from tqdm.auto import tqdm

from tools import get_str_info

In [2]:
# define the path to tif files
path_1 = 'D:\GEE_tif'
path_2 = "N:\LUF-Modelling\LUTO2_JZ\Built_map\GEE_tif_l"

# get all tif files
tifs = glob(f'{path_1}/*.tif') + glob(f'{path_2}/*.tif')

# convert tif_paths to a df, append name columns
files_df = pd.DataFrame({'path':tifs})
files_df[['region','type','year']] = files_df['path'].apply(lambda x:get_str_info(x)).tolist()

# append the tif_paths of same {region|type|year} as a list
grouped_df = files_df.groupby(['region','type','year']).apply(lambda x:x['path'].tolist()).reset_index()

In [7]:
# save the tifs of of same {region|type|year} as a virtual raster
for _,row in tqdm(grouped_df.iterrows(), total=len(grouped_df)):
    
    vir = gdal.BuildVRT(f"../out_data/{row['region']}_{row['type']}_{row['year']}.vrt", row[0])
    vir.FlushCache()

  0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
# save the transformation info
VRT_files = glob('../out_data/*.vrt')

# convert VRT files into df
tif_info = []
for i in tqdm(VRT_files,total=len(VRT_files)):
    region,tif_type,year = get_str_info(i) 
    tif_meta = rasterio.open(i).meta
    tif_info.append({'region':region,'type':tif_type,'year':year,'trans':tif_meta})

# put all record into a df
tif_df = pd.DataFrame(tif_info)

# save tif_df to disk
tif_df.to_pickle('../out_data/tif_df.pkl')

  0%|          | 0/204 [00:00<?, ?it/s]